### Install Python Libraries

In [ ]:

%pip install openai langchain redis tiktoken

### Configure LLM Models

- Import libraries
- Configure access information and paths
- Configure model parameters
- Set Redis connection
- Configure Azure Cache for Redis to be used as a semantic cache

In [43]:
import openai
import redis
import os
import langchain
from langchain.llms import AzureOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.globals import set_llm_cache
from langchain.cache import RedisSemanticCache
import time


AZURE_ENDPOINT="<azure-openai-endpoint>"
API_KEY="<azure-openai-key>"
API_VERSION="2023-05-15"
LLM_DEPLOYMENT_NAME="<name-of-your-gpt-deployment>"
LLM_MODEL_NAME="gpt-35-turbo-instruct"
EMBEDDINGS_DEPLOYMENT_NAME="<name-of-your-embeddings-deployment>"
EMBEDDINGS_MODEL_NAME="text-embedding-ada-002"

REDIS_ENDPOINT = "<azure-cache-for-redis-endpoint-name>" # must include port at the end. e.g. "redisdemo.eastus.redisenterprise.cache.azure.net:10000"
REDIS_PASSWORD = "<azure-cache-for-redis-password>"

# make sure you have an LLM deployed in your Azure Open AI account. In this example, I used the GPT 3.5 turbo instruct model. My deployment was named "gpt35instruct".
llm = AzureOpenAI(
    deployment_name=LLM_DEPLOYMENT_NAME,
    model_name="gpt-35-turbo-instruct",
    openai_api_key=API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    openai_api_version=API_VERSION,
)
# make sure you have an embeddings model deployed in your Azure Open AI account. In this example, I used the text embedding ada 002 model. My deployment was named "textembedding".
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
    model="text-embedding-ada-002",
    openai_api_key=API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    openai_api_version=API_VERSION
)

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

# set up the semantic cache for your llm
set_llm_cache(RedisSemanticCache(redis_url = redis_url, embedding=embeddings, score_threshold=0.01))

#note: you can use score_threshold to change how sensitive the semantic cache is. The lower the score, the less likely it is to use a cached result.

### Run the LLM
Try runnning again with different queries to see what is cached and what is not.

In [44]:
%%time
response = llm("Write a poem about cute puppies.")
print(response)



Oh, little balls of fluff and fur
With wagging tails and tiny paws
Puppies, oh puppies, so pure
The epitome of cuteness, no flaws

With big round eyes that melt our hearts
And floppy ears that bounce with glee
Their playful antics, like works of art
They bring joy to all they see

Their soft, warm bodies, so cuddly
As they curl up in our laps
Their gentle kisses, so lovingly
Like tiny, wet, puppy taps

Their clumsy steps and wobbly walks
As they explore the world anew
Their curiosity, like a ticking clock
Always eager to learn and pursue

Their little barks and yips so sweet
Fill our days with endless delight
Their unconditional love, so complete
Makes everything feel just right

Oh, how can one resist their charms
As they snuggle close and nuzzle our face
With their innocent eyes and wagging arms
They fill our hearts with endless grace

Puppies, oh puppies, so precious and dear
Forever in our hearts, they'll stay
For they bring us love and laughter, year after year
Our cute little p